![LuaLogo.png](LuaLogo.png)

# Introduction to Proc Lua

## Introduction

*"Programming in SAS® has just been made easier" ... "Lua offers you a fresh way to write SAS programs" ...*<br>
... as Paul Tomas, Proc Lua developer at SAS Institute Inc., writes in his paper [Driving SAS® with Lua](https://support.sas.com/resources/papers/proceedings15/SAS1561-2015.pdf) (SAS Global Forum, 2015)
<br><br>
*"The Lua language seems likely to play an increasing role in the SAS world"*<br>
... as Amadeus Software write [here](https://amadeus.co.uk/sas-tips/using-lua-instead-of-sas-macro-language/).

### What is Lua anyway?

- Lua is a "powerful, efficient, lightweight, embeddable scripting language" (as Lua describes itself)
- Lua was developed by Roberto Ierusalimschy (et al) at the Pontifical Catholic University of Rio de Janeiro in Brazil
- Lua is Portuguese for "moon"

### Why Lua?

#### Paraphrased from [A Comparison of the LUA Procedure and the SAS Macro Facility (Vijayaraghavan, 2017)](https://support.sas.com/resources/papers/proceedings17/SAS0212-2017.pdf):

- Need for an alternative to Macro language were felt by various solutions groups at SAS due to the inherent limitations of the latter
- Not feasible to enhance Macro language to the level of a modern scripting language like Lua
- Purpose of Lua is to script C-based software and SAS is written in C, so the two are a good fit for each other
- Lua provides superior debugging information
    
#### What else?

- Proc Lua provides an implementation of Lua 5.2 within Base SAS® (from 9.4)
- Low "entry requirements" for you as a SAS programmer:
    * User-friendly syntax, gentle learning curve
    * Direct access to the vast majority of SAS functions
    * Proc Lua is a brand new way to generate SAS code and provides a realistic and powerful alternative to the SAS Macro language and `call execute()`
- Reputation for performance and memory efficiency
- Support for highly flexible data structures

### First look at Proc Lua

```lua
-- Line comments in Lua start with two dashes
--[[
   And this is a
   block comment
]]
```

```
PROC LUA <restart|terminate>;
    SUBMIT;
       -- Lua code goes inside a SUBMIT block (but not always)
       -- Semicolons are optional in Lua and are usually omitted
       -- Lua is case sensitive
    ENDSUBMIT;
RUN;
```

### Lua basics

#### Data types

- Lua is dynamically typed
- Types are: number, string, boolean, table, nil, function, userdata and thread
- `nil` represents absence of a value, and is different from a SAS missing value
- Boolean values are `true` and `false`
    * **Only** `nil` and `false` evaluate to `false`
    * **Everything** else evaluates to `true` (including zero and SAS missing values!)

#### Naming rules

Same as SAS, except: can be as long as you like (subject to GPP)

#### Declaring variables

Variables have global scope within the current Lua state unless explicitly declared as local

```
local pi
pi = 3.1415926

local pi = 3.1415926
local v1, v2 = 'Hello', 'World'   -- Note list-style declaration and value assignment
```

#### Writing to the log

```
print(v1..', '..v2..'!')          -- Writes "Hello, World!"
```

In [36]:
PROC LUA;
    SUBMIT;
        local v1, v2 = 'Hello', 'World'
        print(v1..', '..v2..'!')
    ENDSUBMIT;
RUN;

### Operators

#### Relational

`<` `>` `<=` `>=` `==` `~=`

#### String

`..` (concatenation)<br>
`#` (length of string or table e.g. #'Hello' returns 5)

#### Arithmetic

`+`  (addition)<br>
`–`  (subtraction, negation)<br>
`*`  (multiplication)<br>
`/`  (division)<br>
`^`  (exponentiation)<br>
`%`  (modulus)

#### Logical

`and`  `or`  `not`<br>
The `and` operator returns its first argument if false, otherwise its second argument<br>
The `or` operator returns its first argument if true, otherwise its second argument

### SAS missing values

- Represented in Proc Lua by `sas.MISSING`
- Evaluates to `true` (we already knew that)

#### Convert to SAS missing value when `nil`
`<expression|variable> or sas.MISSING`

### Basic control

#### if-then-else

```lua
if i == 1 then
   -- Conditional code
elseif i == 2 then
   -- More conditional code
else
   -- Yet more conditional code
end
```


#### Loops

```lua
for i = 0, 15, 5 do
   print(i)
end

local i = 0
repeat
   print(i)
   i = i + 5
until (i > 15)

local i = 0
while (i <= 15) do
   print(i)
   i = i + 5
end
```

In [2]:
PROC LUA;
    SUBMIT;
        for i = 1, 3 do
            if i == 1 then
                print(i..': Start of loop')
            elseif i == 2 then
                print(i..': Middle of loop')
            else
                print(i..': End of loop')
            end
        end
        print(i)
    ENDSUBMIT;
RUN;

### Tables

- Tables are fundamental to Lua
- Tables form the basis for highly flexible and customisable data structures, similar to lists in SCL
- Tables can contain items of different type (unlike SAS arrays)

#### Tables as arrays or lists


In [3]:
PROC LUA;
    SUBMIT;
        colours = {'red', 'blue', 'green', 'yellow'}
        print(colours[1])
    ENDSUBMIT;
RUN;

#### Using `ipairs()` to read through a list

In [4]:
PROC LUA;
    SUBMIT;
        -- local colours = {'red', 'blue', 'green', 'yellow'}
        for i, colour in ipairs(colours) do
            print(i, colour)
        end
    ENDSUBMIT;
RUN;

#### Lua tables as hash tables (key-value pairs)

In [5]:
PROC LUA;
    SUBMIT;
        sp_domains = {CO='Comments',
                      DM='Demographics',
                      SE='Subject Elements',
                      SV='Subject Visits'}
        print(sp_domains.CO)
        print(sp_domains['DM'])
    ENDSUBMIT;
RUN;

Hash table values can be accessed via dot notation or \[*key-as-literal-or-variable-or-expression*]

#### Using `pairs()` to read through a hash table

In [6]:
PROC LUA;
    SUBMIT;
        --[[
        local sp_domains = {CO='Comments',
                            DM='Demographics',
                            SE='Subject Elements',
                            SV='Subject Visits'}
        ]]
        for code, decode in pairs(sp_domains) do
            print(code, decode)
        end
    ENDSUBMIT;
RUN;

## Lua and SAS

### Submitting SAS code

#### Ways to run SAS code

Two functions:

```lua
sas.submit   -- Submits immediately (restricted buffer size: ~32k)
sas.submit_  -- Delays submission until sas.submit() is encountered
```

In [7]:
PROC LUA;
    SUBMIT;
        sas.submit('proc print data=sashelp.class; where age = 12; run;')
    ENDSUBMIT;
RUN;

Obs,Name,Sex,Age,Height,Weight
6,James,M,12,57.3,83.0
7,Jane,F,12,59.8,84.5
10,John,M,12,59.0,99.5
13,Louise,F,12,56.3,77.0
16,Robert,M,12,64.8,128.0


In [8]:
PROC LUA;
    SUBMIT;
        local mycode = 'proc print data=sashelp.class; where age = 13;'
        sas.submit_(mycode)
        print('Waiting for submit()...')
        sas.submit('run;')
    ENDSUBMIT;
RUN;

Obs,Name,Sex,Age,Height,Weight
2,Alice,F,13,56.5,84
3,Barbara,F,13,65.3,98
9,Jeffrey,M,13,62.5,84


In [9]:
PROC LUA;
    SUBMIT;
        sas.submit
        [[
          proc print data=sashelp.class;
              where age = 14;
          run;
        ]]
    ENDSUBMIT;
RUN;

Obs,Name,Sex,Age,Height,Weight
1,Alfred,M,14,69.0,112.5
4,Carol,F,14,62.8,102.5
5,Henry,M,14,63.5,102.5
12,Judy,F,14,64.3,90.0


In [10]:
PROC LUA;
    SUBMIT;
        local code = [[
                       proc print data=sashelp.class;
                           where age = 16;
                       run;
                     ]]
        sas.submit(code)
    ENDSUBMIT;
RUN;

Obs,Name,Sex,Age,Height,Weight
15,Philip,M,16,72,150


#### Substitution

`sas.submit` and `sas.submit_` take a table of key-value pairs as a second argument for resolution in the SAS code

In [11]:
PROC LUA;
    SUBMIT;
        sas.submit('proc print data=sashelp.class; where age = @age@; run;', {age=12})
    ENDSUBMIT;
RUN;

Obs,Name,Sex,Age,Height,Weight
6,James,M,12,57.3,83.0
7,Jane,F,12,59.8,84.5
10,John,M,12,59.0,99.5
13,Louise,F,12,56.3,77.0
16,Robert,M,12,64.8,128.0


In [12]:
PROC LUA;
    SUBMIT;
        local substit = {var='ag'..'e', val=10+3}
        local procprint = 'proc print data=sashelp.class; where @var@ = @val@; run;'
        sas.submit(procprint, substit)
    ENDSUBMIT;
RUN;

Obs,Name,Sex,Age,Height,Weight
2,Alice,F,13,56.5,84
3,Barbara,F,13,65.3,98
9,Jeffrey,M,13,62.5,84


In [13]:
PROC LUA;
    SUBMIT;
        sas.submit
        ([[
          proc print data=@ds@;
              where age = 14;
          run;
        ]], {ds='sashelp.class'})
    ENDSUBMIT;
RUN;

Obs,Name,Sex,Age,Height,Weight
1,Alfred,M,14,69.0,112.5
4,Carol,F,14,62.8,102.5
5,Henry,M,14,63.5,102.5
12,Judy,F,14,64.3,90.0


In [14]:
PROC LUA;
    SUBMIT;
        local var, val = 'age', 15
        local procprint = 'proc print data=sashelp.class; where @var@ = @val@; run;'
        sas.submit(procprint, substit)
    ENDSUBMIT;
RUN;

Obs,Name,Sex,Age,Height,Weight
8,Janet,F,15,62.5,112.5
14,Mary,F,15,66.5,112.0
17,Ronald,M,15,67.0,133.0
19,William,M,15,66.5,112.0


The variables for substitution must be local for this to work

#### Other ways to pass values into Lua I

Via the `SUBMIT` statement: `SUBMIT <"assignment(s);">;`<br>
Macro variables are not recognised within Lua and will not be resolved!

In [15]:
%let gr1 = Hello;
%let gr2 = how are you?;
PROC LUA;
    SUBMIT "greeting = '&gr1'";          /* greeting is global */
        print(greeting..', '.."&gr2")
    ENDSUBMIT;
RUN;

### Using SAS functions in Lua

- The vast majority of SAS functions are available in Proc Lua, just add `sas.` to their function names like this: `sas.sum` `sas.scan` `sas.prxmatch`
- Functions which do not make sense outside of a data step are not available e.g. the `lag` function
- Note that with `sas.put` the format must be a string, variable or expression because Lua doesn't recognise a SAS format as a format
- Functions created by Proc FCMP are similarly called using the `sas.` prefix
- Proc Lua also provides a set of special use functions including dataset and table handling functions

In [16]:
PROC LUA;
    SUBMIT;
        local var = 'Hello Goodbye'
        print('>>>', sas.scan(var,1))
    ENDSUBMIT;
RUN;

#### Other ways to pass values into Lua II

`sas.symget('<macro_var>')`


In [17]:
%let text = Hello, how are you?;
PROC LUA;
    SUBMIT;
        print(sas.symget('text'))
    ENDSUBMIT;
RUN;

#### sas.glibname and sas.gfilename

- If you use sas.libname to allocate a SAS library from within Proc Lua the library will only be accessible by Lua
- Use sas.glibname for global allocation of libraries
- The same goes for sas.filename

### Reading SAS datasets

#### First, let's see how to check for the existence of a dataset

In [18]:
PROC LUA;
    SUBMIT;
        local ds = 'SASHELP.IRIS'
        if sas.exist(ds) then
            print(ds..' exists!')
        else
            print(ds..' does not exist!')
        end
    ENDSUBMIT;
RUN;

In [19]:
PROC LUA;
    SUBMIT;
        local ds = 'SASHELP.XXX'
        if sas.exist(ds) then
            print(ds..' exists!')
        else
            print(ds..' does not exist!')
        end
    ENDSUBMIT;
RUN;

What went wrong?

In [20]:
PROC LUA;
    SUBMIT;
        local ds = 'SASHELP.XXX'
        print('>>>', sas.exist(ds))
    ENDSUBMIT;
RUN;

In [21]:
PROC LUA;
    SUBMIT;
        local ds = 'SASHELP.XXX'
        print('>>>', sas.exists(ds))
        if sas.exists(ds) then
            print(ds..' exists!')
        else
            print(ds..' does not exist!')
        end
    ENDSUBMIT;
RUN;

*SOLVED!!*<br>
`sas.exists` is a special function which returns a boolean value

In [22]:
PROC LUA;
    SUBMIT;
        local ds = 'SASHELP.XXX'
        if sas.exist(ds) > 0 then
            print(ds..' exists!')
        else
            print(ds..' does not exist!')
        end
    ENDSUBMIT;
RUN;

Of course you can force Lua to return a Boolean, as in the above ↑

#### Reading a SAS dataset I

Iterate over whole dataset, load all variables one observation at a time

In [23]:
PROC LUA;
    SUBMIT;
        local dsid = sas.open('sashelp.class')
        for obs in sas.rows(dsid) do
            print(obs.name..' is '..obs['age']..' years old.')
        end
        sas.close(dsid)
    ENDSUBMIT;
RUN;

Here we see the use of one of the dataset handling functions provided in Proc Lua `sas.rows`<br>
This approach is good for "narrow" tables with few columns because for each observation all variables are loaded into a Lua table (here called `obs`) whether subsequently needed or not

#### Reading a SAS dataset II

Iterate over whole dataset, load specific variables one observation at a time

In [24]:
PROC LUA;
    SUBMIT;
        local dsid = sas.open('sashelp.class')
        while sas.next(dsid) do
            print(sas.get_value(dsid,'name')..' is '..sas.get_value(dsid,'age')..' years old.')
        end
        sas.close(dsid)
    ENDSUBMIT;
RUN;

Here we see two further dataset handling functions in action: `sas.next` and `sas.get_value`. The former allows us to read through a SAS dataset from start to finish, and the latter provides access to specific variables, either by name or position. This approach overcomes the performance disadvantage of reading entire observations into Lua.

#### Reading a SAS dataset III

Load entire dataset (not a good idea for large datasets)

In [25]:
PROC LUA;
    SUBMIT;
        local ds = sas.load_ds('sashelp.class')
        for i = 1, table.size(ds) - 3 do 
            print(ds[i]['name']..' is '..ds[i]['age']..' years old.')
        end
    ENDSUBMIT;
RUN;

New in this example:
- `sas.load_ds` (alias: `sas.read_ds`) which loads an entire dataset into a Lua table (returns `nil` if the dataset does not exist)
- `table.size` which returns the number of elements in a table (the first table handling function we have seen)
- Concept of nested tables

##### Nested tables

In [26]:
PROC LUA;
    SUBMIT;
        local ds = sas.load_ds('sashelp.class')
        print('Table "ds" has '..table.size(ds)..' elements.')
        print(table.tostring(ds))
    ENDSUBMIT;
RUN;

In this example DS has 22 elements:
- 19 elements for observations (one table per observation)
- 1 element containing number of variables
- 1 element containing nested table of metadata (one table per variable)
- 1 element containing dataset name

Note `table.tostring`, another table handling function, which dumps the contents of a table to the log

In [27]:
PROC LUA;
    SUBMIT;
        local ds = sas.load_ds('sashelp.class')
        print('Dataset "'..ds['name']..'" has '..ds['nvars']..' variables.')
        print('Variable "sex" has a length of '..ds['vars'].sex['length']..'.')
    ENDSUBMIT;
RUN;

With `ds['vars'].sex['length']` we are reaching down three levels<br>
Note use here of mixture of square-bracket and dot notation (not necessarily recommended)

### Writing SAS datasets

#### Creating a new dataset

1. Use function `sas.new_table(<dataset-name>, <table-of-variable-metadata-tables>)` to define the new dataset
1. Open the dataset in update mode using `sas.open(<dataset-name>, 'u')`
1. Use `sas.append(<dsid>)` to append a new (but empty) observation
1. Use `sas.put_value(<dsid>, <varname>, <value>)` to populate variables
1. Use `sas.update(<dsid>)` to "commit" the observation to the dataset
1. Close the dataset with `sas.close(<dsid>)`

In [28]:
PROC LUA;
    SUBMIT;
        sas.new_table('squares', {
                {name='n',  type='n', length=8, label='N'},
                {name='n2', type='n', length=8, label='N squared'},
        })
        local dsid = sas.open('squares', 'u')
        for n = 1, 10 do
            sas.append(dsid)
            sas.put_value(dsid, 'n', n)
            sas.put_value(dsid, 'n2', n^2)
            sas.update(dsid)
        end;
        sas.close(dsid)
        sas.submit [[
            proc print data=squares noobs label;
            run;
        ]]
    ENDSUBMIT;
RUN;

N,N squared
1,1
2,4
3,9
4,16
5,25
6,36
7,49
8,64
9,81
10,100


#### Updating an existing dataset (adding new observations)

- As above, from 2.

#### Updating an existing dataset (updating existing observations)

1. Open the dataset to update in update mode using `sas.open(<dataset-name>, 'u')`
2. If required, apply where-clause using `sas.where(<dsid>, "<where-clause>")` to isolate the observation(s) to update
3. Use `sas.next(<dsid>)` to move to the first observation (which fulfils the where-clause if applied)
4. Use `sas.put_value(<dsid>, <varname>, <value>)` to update variables
5. Use `sas.update(<dsid>)` to "commit" the observation to the dataset
6. Close the dataset with `sas.close(<dsid>)`

### Running Proc Lua inside SAS macros

- Embedding Lua code inside Proc Lua within a SAS macro is prohibited by the internal handling of SUBMIT/ENDSUBMIT blocks
- However, external Lua files CAN be referenced by Proc Lua via the `INFILE` option
- The external code referenced contains what *would* come within the SUBMIT/ENDSUBMIT block
- The `LuaPath` filename tells Proc Lua where to look for Lua files
- External Lua files have the extension `.lua` however the file is referenced without extension:

```
FILENAME LuaPath '/my/path/luafiles';
PROC LUA INFILE='myluafile';
RUN;
```

- Example below

```
%MACRO _set_graphoption(
    dsname    = _graphopts
  , plot      = .
  , cell      = .
  , object    =
  , type      =
  , attribute =
  , value     =
  , noset     =
  , module    =
);

    %if %sysfunc(prxmatch(/[de]/i,&noset)) %then %do;
        %if %sysfunc(prxmatch(/d/i,&noset)) and %sysfunc(prxmatch(/<default>/i,&value)) %then %return;
        %if %sysfunc(prxmatch(/e/i,&noset)) and (%length(&value) = 0 or &value = ()) %then %return;
    %end;

    proc lua infile='_set_graphoption';
    run;

%MEND _set_graphoption;
```

```lua
function get_parm(var)
    return sas.symget(var) or sas.MISSING
end

function put_vals(id, p)
    for n, v in pairs(p) do
        sas.put_value(id, n, v)
    end
end

local parms = {plot      = get_parm('plot'),
               cell      = get_parm('cell'),
               type      = get_parm('type'):lower(),
               object    = get_parm('object'):lower(),
               attribute = get_parm('attribute'):lower(),
               value     = get_parm('value')
              }
local dsname = sas.symget('dsname')
local dsid

if sas.exists(dsname) then
    local where = string.format('object="%s" and attribute="%s" and plot=%s and cell=%s', parms.object, parms.attribute, parms.plot, parms.cell)
    dsid = sas.open(dsname, 'u')
    sas.where(dsid, where)
    if sas.next(dsid) == nil then
        sas.append(dsid)
    end
    put_vals(dsid, parms)
    sas.update(dsid)
    sas.close(dsid)
else
    sas.new_table(dsname, {
            {name='plot',      type='n', length=8,    label='Plot ID'},
            {name='cell',      type='n', length=8,    label='Cell ID'},
            {name='type',      type='c', length=200,  label='Graph Object Type'},
            {name='object',    type='c', length=200,  label='Graph Object Name'},
            {name='attribute', type='c', length=200,  label='Graph Object Attribute'},
            {name='value',     type='c', length=1000, label='Attribute Value'}
        }
    )
    dsid = sas.open(dsname, 'u')
    sas.append(dsid)
    put_vals(dsid, parms)
    sas.update(dsid)
    sas.close(dsid)
end
```

## Defining functions

- Functions are defined as follows:

```lua
<local> function function-name(arg1, arg2, arg3...)
    body of function
    return comma-separated-list-of-values
end
```

- Lua functions can return multiple values, including tables
- (Advanced: Functions can be assigned to variables and hence be passed as an argument to other routines or be returned as the result of a function)
- Here's a simple function which splits a SAS date into day, month and year:

```lua
function dateparts(sasdate)
    return sas.day(sasdate), sas.month(sasdate), sas.year(sasdate)
end
```

- Let's try it out

In [35]:
PROC LUA;
    SUBMIT;
        local function dateparts(sasdate)
            return sas.put(sas.day(sasdate),'z2'), sas.put(sas.month(sasdate),'z2'), sas.year(sasdate)
        end
        local d, m, y = dateparts(sas.today())
        print("Today's date is: "..y..m..d)
    ENDSUBMIT;
RUN;

### Macro quoting

#### Quoting disaster (no disaster yet)

In [30]:
%macro quoting_disaster(p);
    %let p = %sysfunc(compbl(&p));
    &p
%mend;

%put >>> %quoting_disaster(a  bb   ccc);

#### Quoting disaster...

In [31]:
%put >>> %quoting_disaster(a,  bb,   ccc);
%put >>> %quoting_disaster();

#### Quoting disaster (resolved?)

In [32]:
%macro quoting_disaster(p);
    %let p = %sysfunc(compbl(%quote(&p)));       /* <= added %quote */
    &p
%mend;

%put >>> %quoting_disaster(%str(a,  bb,   ccc)); /* <= added %str   */
%put >>> %quoting_disaster();

#### Quoting disaster (resolved!)

In [33]:
PROC LUA;
    SUBMIT;
        function no_quoting_disaster(p)
            return sas.compbl(p)
        end
    
        print('>>> '..no_quoting_disaster('a,  bb,   ccc'))
        print('>>> '..no_quoting_disaster(''))
    ENDSUBMIT;
RUN;

## Some simple file i/o

- Use the `io` module
- Here's an example of file i/o which creates a file, writes some text and reads it back out again

```
PROC LUA;
    SUBMIT;
        local p = 'c:\\temp\\luaiotest.txt'

        -- Create a new file (or overwrite an existing one)
        f = io.open(p, 'w')

        -- Set the default output file to luaiotest.txt
        io.output(f)

        -- Write some text to the file
        io.write('The quick brown fox\njumps over the lazy dog.')

        -- Close the file
        f:close()
    
    
        -- Open our file for reading
        f = io.open(p, 'r')

        -- Set the default input file to our file
        io.input(f)

        -- Read the file using io.lines
        for t in io.lines() do
            print(t)
        end

        -- Close the file
        f:close()
    ENDSUBMIT;
RUN;
```

In [34]:
PROC LUA;
    SUBMIT;
        local p = 'c:\\temp\\luaiotest.txt'

        f = io.open(p,'w')
        io.output(f)
        io.write('The quick brown fox\njumps over the lazy dog.')
        f:close()

        f = io.open(p,'r')
        io.input(f)
        for t in io.lines() do
            print(t)
        end
        f:close()
    ENDSUBMIT;
RUN;

## Limitations of Proc Lua

- Lua code cannot be used directly inside a SAS macro
- Macros defined in Proc Lua are only available within the Lua state
- Not all Lua modules supported (e.g. `os`)
- Third-party Lua modules may not work in Proc Lua (particularly those not written for Lua 5.2)
- Lua's multi-threading capabilities are disabled in Proc Lua
- Current lack of familiarity with Lua in the pharma SAS community
- Prevailing conservatism within the pharma SAS community

## Conclusion

- Proc Lua enables you to combine a modern scripting language with the functionality of SAS
- It's easy to get started with Lua - no steep learning curve to conquer before you can get going
- *"Programming in SAS® has just been made easier"*
- *"Lua offers you a fresh way to write SAS programs"* 
- *"The Lua language seems likely to play an increasing role in the SAS world"*

**HIGHLY RECOMMENDED:**

- Click [here](https://blogs.sas.com/content/sasdummy/2015/08/03/using-lua-within-your-sas-programs/) to hear Paul Tomas interviewed about Proc Lua.
- Click [here](https://sasensei.com/test/87#Lua) for a short Lua quiz - test what you learnt today! (*Registration required for new users*).

## Further reading

- [https://www.lua.org](https://www.lua.org)
- [https://en.wikibooks.org/wiki/Lua_Programming](https://en.wikibooks.org/wiki/Lua_Programming)
- [https://www.tutorialspoint.com/lua/index.htm](https://www.tutorialspoint.com/lua/index.htm)
- [SAS® Help Center: Concepts: PROC LUA](http://documentation.sas.com/?docsetId=proc&docsetTarget=p0t7zanjrat68hn1ue4mmy337rqe.htm&docsetVersion=9.4&locale=en)
- [Execute Lua online](https://www.tutorialspoint.com/execute_lua_online.php)
- [Driving SAS® with Lua (Tomas, 2015)](https://support.sas.com/resources/papers/proceedings15/SAS1561-2015.pdf)
- [A Comparison of the LUA Procedure and the SAS® Macro Facility (Vijayaraghavan, 2017)](https://support.sas.com/resources/papers/proceedings17/SAS0212-2017.pdf)
- [Using Lua instead of macro language](https://amadeus.co.uk/sas-tips/using-lua-instead-of-sas-macro-language/)
- [How to set up Jupyter Notebook for SAS® (Bulana, 2018)](http://www.pharmasug.org/proceedings/2018/BB/PharmaSUG-2018-BB07.pdf)

**Acknowledgments**

Thanks go to my esteemed colleague Igor Khorlo who takes the blame for getting me interested in Lua in the first place, and who is a constant and ever willing source of good advice and valuable feedback.


*SAS and all other SAS Institute Inc. product or service names are registered trademarks or trademarks of SAS Institute Inc. in the USA and other countries. ® indicates USA registration.*

*Lua software: Copyright © 1994–2017 Lua.org, PUC-Rio*

*This presentation: Author Rowland Hale, Copyright © 2018 Syneos Health*

![syneos.png](syneos.png)